In [ ]:
# !rm -fr "/content/food_101" 

In [ ]:
# !gdown 1H1GZ2bzMcnwec8_NTWeBkFtXxyUeVTZ1 

In [ ]:
# !unzip "/content/food_101_3_classes.zip" -d food_101

In [ ]:
# !rm -rf "/content/food_101/__MACOSX"

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [ ]:
train_path = "/content/food_101/food_101_3_classes/train"
test_path = "/content/food_101/food_101_3_classes/test"

In [ ]:
train_gen = ImageDataGenerator(rescale=1/255.)
test_gen = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_set = train_gen.flow_from_directory(train_path,
                                          target_size=(224, 224),
                                          batch_size=32,
                                          class_mode="categorical")

test_set = test_gen.flow_from_directory(test_path,
                                        target_size=(224, 224),
                                        batch_size=32,
                                        class_mode="categorical")

In [ ]:
print("Train set length", len(train_set))

batch = next(train_set)

print("Batch [0] shape", batch[0].shape)
print("Batch [1] shape", batch[1].shape)

img = batch[0][0]
print("Image shape", img.shape)

label = batch[1][0]
print("Label", label)

print("Test set classes", test_set.class_indices)
label_texts = ["Hamburger", "Ice cream", "Pizza"]

plt.figure(figsize=(8, 8))

for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.title(label_texts[np.argmax(batch[1][i])])
    plt.imshow(batch[0][i])

plt.tight_layout()
plt.show()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(10, 3, activation="relu", input_shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(10, 3, activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(10, 3, activation="relu"),
    tf.keras.layers.Conv2D(10, 3, activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3, activation="softmax")
])

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
history = model.fit(train_set,
                    epochs=5,
                    validation_data=test_set,
                    steps_per_epoch=len(train_set),
                    validation_steps=int(.25 * len(test_set)))

In [ ]:
model.evaluate(test_set)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 8))
plt.show()

In [ ]:
test_batch = next(test_set)
predictions = model.predict(test_batch[0])

plt.figure(figsize=(8, 8))

for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.title(label_texts[np.argmax(predictions[i])])
    plt.imshow(test_batch[0][i])

plt.tight_layout()
plt.show()

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(10, 3, activation="relu", input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(10, 3, activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(10, 3, activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3, activation="softmax")
])

model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [ ]:
history2 = model2.fit(train_set,
                      epochs=5,
                      validation_data=test_set,
                      steps_per_epoch=len(train_set),
                      validation_steps=int(.25 * len(test_set)))

In [ ]:
model2.evaluate(test_set)

In [ ]:
pd.DataFrame(history2.history).plot(figsize=(8, 8))
plt.show()

In [ ]:
predictions2 = model2.predict(test_batch[0])

plt.figure(figsize=(8, 8))

for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.title(label_texts[np.argmax(predictions2[i])])
    plt.imshow(test_batch[0][i])

plt.tight_layout()
plt.show()

In [ ]:
image_gen = ImageDataGenerator(rotation_range=40,
                               height_shift_range=0.2,
                               width_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode="nearest")

In [ ]:
import os
from skimage import io
from PIL import Image

In [ ]:
hamburgers = []
hamburger_dir = "/content/food_101/food_101_3_classes/train/hamburger/"
my_images = os.listdir(hamburger_dir)

for i, image_name in enumerate(my_images):
    if (image_name.split(".")[1] == "jpg"):
        image = io.imread(hamburger_dir + image_name)
        image = Image.fromarray(image, "RGB")
        image = image.resize((224, 224))
        hamburgers.append(np.array(image))

hamburgers = np.array(hamburgers)

In [ ]:
print(hamburgers.shape)

plt.imshow(hamburgers[0])
plt.show()

In [ ]:
i = 0
for batch in image_gen.flow(hamburgers,
                            batch_size=32,
                            save_to_dir=hamburger_dir,
                            save_prefix="ham_aug",
                            save_format="jpg"):
    if i == 12:
        break
    i += 1

In [ ]:
def augment_images(img_dir, image_generator):
    dataset = []
    images = os.listdir(img_dir)
    for i, image_name in enumerate(images):
        ext = image_name.split(".")[1]
        if ext != "jpg":
            continue

        image = io.imread(img_dir + image_name)
        image = Image.fromarray(image, "RGB")
        image = image.resize((224, 224))
        dataset.append(np.array(image))

    dataset = np.array(dataset)

    i = 0
    for batch in image_generator.flow(dataset, batch_size=32,
                                      save_to_dir=img_dir, save_prefix="aug",
                                      save_format="jpg"):
        if i == 12:
            break
        i += 1

In [ ]:
augment_images("/content/food_101/food_101_3_classes/train/ice_cream/", image_gen)
augment_images("/content/food_101/food_101_3_classes/train/pizza/", image_gen)

In [ ]:
train_gen = ImageDataGenerator(rescale=1/255.)
test_gen = ImageDataGenerator(rescale=1/255.)

train_set = train_gen.flow_from_directory(train_path,
                                          target_size=(224, 224),
                                          batch_size=32,
                                          class_mode="categorical")

test_set = test_gen.flow_from_directory(test_path,
                                        target_size=(224, 224),
                                        batch_size=32,
                                        class_mode="categorical")

In [ ]:
print("Train set length", len(train_set))

batch = next(train_set)

print("Batch [0] shape", batch[0].shape)
print("Batch [1] shape", batch[1].shape)

img = batch[0][0]
print("Image shape", img.shape)

label = batch[1][0]
print("Label", label)

print("Test set classes", test_set.class_indices)
label_texts = ["Hamburger", "Ice cream", "Pizza"]

plt.figure(figsize=(8, 8))

for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.title(label_texts[np.argmax(batch[1][i])])
    plt.imshow(batch[0][i])

plt.tight_layout()
plt.show()

In [ ]:
model3 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(10, 3, activation="relu", input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(10, 3, activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(10, 3, activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3, activation="softmax")
])

model3.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

history3 = model3.fit(train_set,
                      epochs=5,
                      steps_per_epoch=len(train_set),
                      validation_data=test_set,
                      validation_steps=int(.25 * len(test_set)))

In [ ]:
model3.evaluate(test_set)

In [ ]:
pd.DataFrame(history3.history).plot(figsize=(8, 8))
plt.show()